In [4]:
import tensorflow as tf
import numpy as np

In [46]:
def mat_pow(a, k):
    if k == 0:
        m = np.eye(a.shape[0])
    elif k % 2:
        m = mat_pow(a, k-1)
        m = tf.matmul(a, m)
    else:
        m = mat_pow(a, k // 2)
        m = tf.matmul(m, m)
    return m

In [357]:
def check_connectivity_tf(mat_tf):
    n, m = mat_tf.shape
    assert n.value == m.value
    mat_tf = tf.linalg.set_diag(mat_tf, tf.ones(n.value, dtype=tf.float64))
    mat_tf = mat_pow(mat_tf, n.value)
    return tf.cast(tf.reduce_sum(tf.cast\
                 (tf.equal(tf.reduce_sum(tf.cast\
                 (tf.not_equal(mat_tf, 0), tf.float64), axis=0), n.value) , tf.float64))\
                 > 0, tf.float64)
def check_no_cycle_tf(mat_tf):
    n, m = mat_tf.shape
    assert n.value == m.value
    a = tf.cast(tf.equal(tf.reduce_sum(mat_tf, axis=0), 1), tf.float64)
    b = tf.cast(tf.equal(tf.reduce_sum(mat_tf, axis=1), 1), tf.float64)
    return tf.cast(tf.equal(tf.reduce_sum(a), n.value), tf.float64) *\
           tf.cast(tf.equal(tf.reduce_sum(b), n.value), tf.float64)
def check_diagonal_zero_tf(mat_tf):
    return tf.reduce_sum(tf.diag_part(mat_tf))

def check_connectivity(mat):
    n, m = mat.shape
    assert n == m
    mat = mat * 1 
    diag = range(n)
    mat[diag, diag] = 1 
    mat = np.linalg.matrix_power(mat, n); 
    #print(mat)
    print(np.sum((mat != 0).sum(axis=0) == m) > 0)
    
def check_no_cycle(mat):
    n, m = mat.shape
    assert n == m
    a = mat.sum(axis=0) == 1
    b = mat.sum(axis=1) == 1
    print((a.sum() == n) & (b.sum() == n))

In [365]:
tf.reset_default_graph()
N = 5

mat = tf.placeholder(shape=[N, N], dtype=tf.float64)
#mat = tf.nn.relu(tf.matmul(dist, W1) + b1)
mat = tf.where(mat > 0, tf.ones(shape=[N, N], dtype=tf.float64), mat)
pat = check_connectivity_tf(mat)
cat = check_no_cycle_tf(mat)
dia = check_diagonal_zero_tf(mat)

In [366]:
a = np.array([[0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 0, 0], [0, 0,     0, 0, 1], [0, 0, 0, 1, 0]])
b = np.array([[0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [0, 0, 0, 0, 1], [1, 0, 0, 0, 0], [0, 1, 0, 0, 0]])
c = np.array([[0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [0, 0, 0, 0, 1], [1, 0, 0, 0, 0], [0, 1, 0, 0, 0]])
d = np.array([[0,0,0,0,1],[0,0,1,0,0],[0,1,0,0,0],[1,0,0,0,0],[0,0,0,1,0]])
my_dist = np.array([[0, 10, 5, 2, 8], [0, 0, 6, 5, 4], [0, 0, 0, 9, 2], [0, 0, 0, 0, 7], [0, 0, 0, 0, 0]])
my_dist += my_dist.T
my_dist

array([[ 0, 10,  5,  2,  8],
       [10,  0,  6,  5,  4],
       [ 5,  6,  0,  9,  2],
       [ 2,  5,  9,  0,  7],
       [ 8,  4,  2,  7,  0]])

In [371]:
#check_connectivity(a)
#check_no_cycle(a)
#check_connectivity(b)
#check_no_cycle(b)
#check_connectivity(c)
#check_no_cycle(c)
check_connectivity(d)
check_no_cycle(c)

False
True


In [370]:
init = tf.global_variables_initializer()
with tf.Session() as sess:
    init.run()
    orig, mardas, raghas, pertas = sess.run([mat, pat, cat, dia], feed_dict = {mat: d})
    print(orig, mardas, raghas, pertas)
    #orig, mardas, raghas, pertas = sess.run([mat, pat, cat, dia], feed_dict = {dist: b})
    #print(orig, mardas, raghas, pertas)
    #orig, mardas, raghas, pertas = sess.run([mat, pat, cat, dia], feed_dict = {dist: c})
    #print(orig, mardas, raghas, pertas)

[[0. 0. 0. 0. 1.]
 [0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0.]] 0.0 1.0 0.0
